In [1]:
##########################################################
## Générateur de librairies VHDL - FPGA AMD             ##
##  -> Génération des spiking trace des 10000 tests     ##
##     MNist et écriture des fichiers de stockge        ##
##     NPZ et JSON                                      ##
## !! Vérification pas deux spikes en suivant sur    !! ##
## !! deux images consécutives                       !! ##
## !!          -> XOR (img[t-1], imgt[t]             !! ##
## !! ajout de 4 bits pour le chiffre à trouver      !! ##
## !! ajout de 2 bits pour la µmachine               !! ##
## !! remplacement du spiking trace 220 par l'image  !! ##
## !! source                                         !! ##
##                                                      ##
## !! MODIFICATION IMPORTANTE DU CODE  ..            !! ##
## !!          MNISTWriteFileSptXOR(50)-01              ##
##########################################################
##          MNISTWriteFileSptXOR(70-µmac)-01            ##
##          --------------------------------            ##
##                                       07-08-2025     ##
## Programme générateur de tous les spiking trace des   ##
## 10000 tests MNIST afin de réaliser une comparaison   ##
## complète entre les différents modèle (Florent - PH)  ##
##                                                      ##
## -> sortie dans 1 fichier NPZ -> 10000 NPY            ##
## -> chaque NPY -> le numéro d'odre MNIST              ##
##               -> la réprésentation complète 28px28p  ##
##               -> les 220 réprésentations spiking t.  ##
##                                                      ##
## -> le programme prend en entrée un fichier           ##
## correctement formaté (utiliser MNISTWriteFIleSpt-1)  ##
## de type npy (il contiend 220 matrices de 28 par      ##
## 28 bit) et génère deux fichiers :                    ##
##   -> un npy de 220 matrices de 28 par 28 -> XOR      ##
##   -> un COE de 220 matrices de 28 par 28 -> XOR      ##
## l'utilisation de la fonction XOR entre deux matrices ##
## permet d'éviter deux spike à '1' consécutifs         ##
##                                                      ##
## tous les fichiers Npy source dans ../*/npy           ##
## tous les fichiers Npy de sortie dans ../*/npyxor     ##
## tous les fichiers COE de sortie dans ../*/coexor     ##
##                                                      ##
## * => racine de base ex /MNIST01 -> à spécifier       ## 
##                                                      ##
##                           Pascal Harmeling (2025)    ##
##########################################################

##########################################################
## PAS DE GRAPHIQUE NI INTERACT lib POUR LA RAPIDITE!   ##
##########################################################

# pour la lecture et le traitement du fichier NPY
using NPZ

using MLDatasets
train_x, train_y = MNIST(split=:test)[:]

(features = Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [3]:
#######################################################################
# à définir pour le début et le nombre d'image crée pour le fichier COE
#######################################################################
#                              Attention
#
# - la taille de la zone => MaxImg
#      il se peut que la BRAM soit trop petite pour contenir le fichier -> reduire MaxImg
# - StartImg+MaxImg<=10000 
# - StSize => toujours '1', '2', '4' ou '5' 
#             - '1' -> 20 black + 200 spicking trace
#             - '2' -> 10 black + 100 spicking trace
#             - '4' ->  5 black +  50 spicking trace
#             - '5' ->  4 black +  40 spicking trace
#             - '10'->  2 black +  20 spicking trace
#
#######################################################################

const StartImg = 1
const MaxImg = 70
const StSize = 1

#################################################
## analyse des valeurs des constantes          ##
#################################################    
if ((StartImg+MaxImg-1)> (last(size(train_x))))
    error("ERROR -> La valeur de (StartImg+MaxImg) est supérieure à la taille max de train_x, arrêt du programme.")
end
 
if ((StSize != 1) && (StSize != 2) && (StSize != 4) && (StSize != 5) && (StSize != 10))
    error("ERROR -> La valeur de StSize doit valoir '1','2','4' ou '5', arrêt du programme.")
end

NbrBlack::Int32 = 20 / StSize
NbrSpiking::Int32 = 200 / StSize

#################################################
## Function lecture du fichier npy             ##
#################################################
# Lire un fichier .npy
a = "./MNIST01/" ## -> à adapter suivant besoin

##constante 
const b = "NPY/"
const c = "NPYXOR/" 
const d = "COEXOR/"
const name = "spiking_number00001.npy" ## supprimer le numéro en fin de programmation !!!!!!!!!!!!!!!!!

# Définir les chemin de travail
dir_pathread = a * b
dir_pathwritenpyxor = a * c
dir_pathwritecoexor = a * d

#################################################
## analyse des chemins et noms de fichiers     ##
#################################################    
if !(isdir(dir_pathread))
    error("ERROR -> Le répertoire de lecture des fichiers n'existe pas -> fin de programme.")
end

if !(isdir(dir_pathwritecoexor))
    error("ERROR -> Le répertoire d'écriture des fichiers n'existe pas -> fin de programme.")
end

#################################################
## création et initialisation des variables de ##
## travail                                     ##
#################################################
XORTSspikes = zeros(Bool, 28,28,220)
array = zeros(Bool, 28,28,220)

#################################################
## programme principal -                       ##
##                   création du fichier *.COE ##
#################################################

## création du fichier COE dans le répertoire COEXOR
filename = dir_pathwritecoexor *"spiking_number_" * lpad(string(StartImg), 5, '0') * "to" * lpad(string(StartImg+MaxImg-1), 5, '0') * "(Size_" * lpad(string(NbrBlack+NbrSpiking),3, '0')* ").coe"
file=open(filename, "w")
write(file, string(";----------------------------------------------------------------------\n"))
write(file, string(";---\n"))
write(file, string(";--- Package producted by julia - creating the Spiking trace matrix XOR\n"))
write(file, string(";---\n"))
write(file, string(";----------------------------------------------Pascal Harmeling 2025---\n"))
write(file, string(";\n"))
write(file, string("memory_initialization_radix=2; binary \n"))
write(file, string("memory_initialization_vector=\n"))

global Img

## boucle principale pour le parcour des MaxImg en commencant en  StartImg
for  Img in StartImg:(StartImg+MaxImg-1)
    Num = string(bitstring(train_y[Img]))[61:64]
    filename = dir_pathread  *"spiking_number" * lpad(string(Img), 5, '0') * ".npy"
    #println(filename)
    if !(isfile(filename))
        println("ERROR -> $filename")
        error("ERROR -> le fichier *.npy n'existe pas")
    end
    
    array = npzread(filename)
    XORTSspikes = zeros(Bool, 28,28,220)
    XORTSspikes[:,:,(NbrBlack+1)] = array[:,:,21]
    XORTSspikes[:,:,(NbrBlack+NbrSpiking)] = >=(0.5).(train_x[:,:,Img]) ## set last display to check answer

    ##génération spiking trace without two spikes ...
    for i in 2:(NbrSpiking-1)
        XORTSspikes[:,:,(NbrBlack+i)] = (XORTSspikes[:,:,(NbrBlack+i-1)] .⊻  array[:,:,(20+i)] ) .& array[:,:,(20+i)]
    end

    ## write all  Spiking Trace matrix XOR (x,y,Img)
    for z=1:1:NbrBlack+NbrSpiking
        write(file, string(";tableau $z de l'image $Img à pour solution : $(train_y[Img]) \n"))
        ## ajoute les informations pour la micro machine
        if (z==1)
            write(file, string("10"))
        elseif (z==(NbrBlack+NbrSpiking))
            write(file, string("01"))
        else
            write(file, string("00"))
        end
        ##ajoute la valeur du chiffre à retrouver
        write(file, Num)
            
        XORTSspikes[:,:,z]= transpose(XORTSspikes[:,:,z])
        for y=28:-1:1
            for x=28:-1:1
                if XORTSspikes[y,x,z]==false
                    write(file, string("0"))
                else
                    write(file, string("1"))
                end
            end
        end
        if (!((z==(NbrBlack+NbrSpiking)) && (Img==(StartImg+MaxImg-1))))
            write(file, string(",\n"))
        end
    end
end
close(file)
println(filename)


./MNIST01/NPY/spiking_number00070.npy
